In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
from nltk import word_tokenize
import pandas as pd
from urllib.request import urlopen
import requests

### Import list of names from json file:

In [2]:
CharacterNames = []
with open('characters.json') as json_file:
    data = json.load(json_file)
    for p in data['characters']:
        CharacterNames.append('characterName')

### Get relevant paths

In [3]:
def GetUrls(url):

    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    
    AllLinks = []
    for link in soup.find_all('a'):
        AllLinks.append(link.get('href'))
    
    idx = AllLinks.index('/wiki/Season_8')
    
    AllLinks = AllLinks[idx:idx+26]
    
    indices = [AllLinks.index("/wiki/Season_" +str(i)) for i in [8, 7, 6]]
    
    SeaLen = [indices[i+1]-indices[i]-1 for i in range(2)] + [len(AllLinks)-indices[2]-1]
    Seasons = SeaLen[0]*['Season_8'] + SeaLen[1]*['Season_7'] + SeaLen[2]*['Season_6']

    j = 0
    for i in indices: 
        del(AllLinks[i-j])
        j+=1

    return AllLinks, Seasons
    



    



### Get documents

In [4]:
def GetDocuments(root, endpath):
    url = root + endpath
    res = requests.get(url)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    text = soup.find_all(text=True)

    output = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        # there may be more elements you don't want, such as "style", etc.
    ]

    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
            
    return soup, text, output

### Accesss all captions

In [5]:
def GetSectionName(soup):
    AllH = soup.find_all(['h2', 'h3'])
    for header in AllH:
        if header.span is not None: 
            header.span.unwrap()

    AllHeaders = [header.text for header in AllH]
    Pre = AllHeaders.index('Summary')
    Post = AllHeaders.index('Appearances')

    HeadlineName = AllHeaders[Pre+1:Post]
    return HeadlineName

### Access all releveant text (including captions)

In [13]:
def GetText(HeadlineNames):
    HeadlineNames += ['Appearances']

    # Unwrap text
    Texts = soup.find_all(['p','h2', 'h3'])
    for section in Texts: 
        if section is not None:
            if section.a is not None: 
                section.a.unwrap()
            elif section.span is not None: 
                section.span.unwrap()

    #S tore text in list
    Texts = [t.text for t in Texts]

    # Find section indices in Texts
    CaptionIndices = []
    for caption in HeadlineNames:         
        CaptionIndices.append(Texts.index(caption))

    # Store texts according to section
    SectionText = []
    for i in range(len(CaptionIndices)-1):
        SectionText.append(Texts[CaptionIndices[i]+1:CaptionIndices[i+1]])


    return SectionText



In [ ]:
#Initialise dataframe
df = pd.DataFrame(columns = ["season", "caption", "text"])

# Get all urls
url = "https://gameofthrones.fandom.com/wiki/"
EndAllUrls, Seasons = GetUrls(url)

root = 'https://gameofthrones.fandom.com'

AllCaptions = []
AllTexts = []
AllSeasons = []
for endpath, season in zip(EndAllUrls, Seasons):
    
    # Webscrabe
    soup, text, output = GetDocuments(root, endpath)

    # Get section names
    Captions = GetSectionName(soup)

    # Get texts
    SectionText = GetText(Captions)
    
    AllCaptions = AllCaptions + Captions[:-1]
    AllTexts = AllTexts + SectionText
    AllSeasons = AllSeasons + [season] * len(Captions[:-1])
    
    
# Store in dataframe
df['caption'] = AllCaptions
df['text'] = AllTexts
df['season'] = AllSeasons


In [23]:
df

,season,caption,text
0,Season_8,At Winterfell,[Queen Daenerys and Jon march south to Winterf...
1,Season_8,In King's Landing,[Euron Greyjoy sails the Iron Fleet back to Ki...
2,Season_8,At Last Hearth,[Lord Umber (now a Wight) is burned to kill hi...
3,Season_8,At Winterfell,[Jaime stands still while Daenerys speaks. She...
4,Season_8,At the Forge,"[At the Forge, the men are hard at work with t..."
...,...,...,...
117,Season_6,In Oldtown,"[Sam in the Citadel's library., Sam, Gilly, an..."
118,Season_6,Beyond the Wall,[Bran sees the vision of his father and Aunt L...
119,Season_6,At Winterfell,[Melisandre and Jon Snow are in Winterfell's m...
120,Season_6,In Dorne,[Ellaria and the Sand Snakes meet with Olenna ...
